# Task 2

**Student Name: Aleesha Zainab**

**Student ID: 04162013021**

Date: 06/15/2024

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include all the libraries you used in your assignment, e.g.,:



1.   os
2.   numpy
3.   pandas
4.   requests
5.   zipfile
6.   io.BytesIO
7.   sklearn.feature_extraction.text.
8.   CountVectorizer
9.   sklearn.feature_extraction.text.
10.  TfidfVectorizer
11.  gensim.models.KeyedVectors




**Introduction**

In this task, the goal is to generate various feature representations for a collection of job advertisements. Only the job advertisement descriptions will be considered for this task. The task requires generating the following feature representations:

Bag-of-Words Model:

Generate the Count vector representation for each job advertisement description based on a predefined vocabulary. Save the Count vector representations into a file named count_vector.txt

Models Based on Word Embeddings:

Choose embedding language model such as glove Generate two types of feature representations for each job advertisement description using the chosen language model:

1.TF-IDF weighted vector representation.

2.Unweighted vector representation.

In [1]:
import os
import numpy as np
import pandas as pd
import requests
import zipfile
from io import BytesIO
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import KeyedVectors
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
with open('/content/drive/MyDrive/Assignment5 data/preprocessed_data/vocab.txt', 'r') as file:
    vocabulary = file.read().splitlines()

In [4]:
import csv
import os

def read_web_indexes(csv_file_path):
    web_indexes = []

    with open(csv_file_path, mode='r') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip the header row
        for row in reader:
            if row:
                web_indexes.append(row[0])

    return web_indexes

csv_file_path = os.path.join(os.getcwd(), '/content/drive/MyDrive/web_index.csv')  #
web_indexes = read_web_indexes(csv_file_path)

In [5]:

preprocessed_dir = '/content/drive/MyDrive/Assignment5 data/preprocessed_data'
job_descriptions = []
exclude_files = {'vocab.txt', 'count_vectors.txt'}

for filename in os.listdir(preprocessed_dir):
    if filename.endswith('.txt') and filename not in exclude_files:
        with open(os.path.join(preprocessed_dir, filename), 'r') as file:
            job_descriptions.append(file.read().strip())


# Count Vector


In [6]:
vectorizer = CountVectorizer(vocabulary=vocabulary)
count_vectors = vectorizer.fit_transform(job_descriptions)

In [7]:
count_vectors_df = pd.DataFrame(count_vectors.toarray())

In [8]:
count_vectors_txt_path = os.path.join(preprocessed_dir, 'count_vectors.txt')

In [9]:
with open(count_vectors_txt_path, 'w') as f:
    for index, row in count_vectors_df.iterrows():
        webindex = web_indexes[index]
        sparse_representation = row.iloc[1:].to_dict()
        line = f"#{webindex},"
        line += ", ".join([f"{index}:{value}" for index, value in sparse_representation.items()])
        f.write(line + '\n')

print("Count vectors written to preprocessed_data/count_vectors.txt successfully.")

Count vectors written to preprocessed_data/count_vectors.txt successfully.


In [11]:
from google.colab import files

# Download the file
files.download('/content/drive/MyDrive/Assignment5 data/preprocessed_data/count_vectors.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



# Tf-idf

In [12]:
def download_glove(url, save_path):
    response = requests.get(url)
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        z.extractall(save_path)

In [13]:
glove_url = "http://nlp.stanford.edu/data/glove.6B.zip"
glove_dir = "glove.6B"
glove_zip_path = os.path.join(glove_dir, "glove.6B.zip")
glove_file_path = os.path.join(glove_dir, "glove.6B.300d.txt")

In [14]:
if not os.path.exists(glove_dir):
    os.makedirs(glove_dir, exist_ok=True)
    print("Downloading GloVe embeddings...")
    response = requests.get(glove_url, verify=False)
    with open(glove_zip_path, 'wb') as f:
        f.write(response.content)
    with zipfile.ZipFile(glove_zip_path, 'r') as z:
        z.extractall(glove_dir)
    print("GloVe embeddings downloaded and extracted.")
else:
    print("GloVe embeddings already exist.")

GloVe embeddings downloaded and extracted.


In [15]:
def load_glove_model(glove_file):
    embedding_model = {}
    try:
        with open(glove_file, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.split()
                word = parts[0]
                vector = np.array(parts[1:], dtype=np.float32)
                embedding_model[word] = vector
    except FileNotFoundError:
        print(f"Error: GloVe file '{glove_file}' not found.")
    return embedding_model

In [16]:
embedding_model = load_glove_model(glove_file_path)

if embedding_model:
    print("GloVe embeddings loaded successfully.")
else:
    print("Failed to load GloVe embeddings.")

GloVe embeddings loaded successfully.


In [17]:
embedding_model = KeyedVectors.load_word2vec_format('/content/glove.6B/glove.6B.300d.txt', binary=False, no_header=True)

In [18]:
def generate_document_embeddings(descriptions, embedding_model, use_tfidf=False):
    vectorizer = TfidfVectorizer(vocabulary=vocabulary) if use_tfidf else CountVectorizer(vocabulary=vocabulary)
    vectors = vectorizer.fit_transform(descriptions)
    feature_names = vectorizer.get_feature_names_out()

    doc_embeddings = []
    for vector in vectors:
        word_indices = vector.nonzero()[1]
        if use_tfidf:
            tfidf_weights = vector.data
        else:
            tfidf_weights = np.ones(len(word_indices))

        doc_vector = np.zeros(embedding_model.vector_size)
        weight_sum = 0

        for i, word_index in enumerate(word_indices):
            word = feature_names[word_index]
            if word in embedding_model:
                word_vector = embedding_model[word]
                weight = tfidf_weights[i]
                doc_vector += word_vector * weight
                weight_sum += weight

        if weight_sum > 0:
            doc_vector /= weight_sum

        doc_embeddings.append(doc_vector)

    return np.array(doc_embeddings)

In [19]:
unweighted_embeddings = generate_document_embeddings(job_descriptions, embedding_model, use_tfidf=False)
tfidf_weighted_embeddings = generate_document_embeddings(job_descriptions, embedding_model, use_tfidf=True)

# Save Unweighted and Weighted Embeddings

In [20]:
np.savetxt('unweighted_embeddings.csv', unweighted_embeddings, delimiter=',')
np.savetxt('tfidf_weighted_embeddings.csv', tfidf_weighted_embeddings, delimiter=',')

# Task 3

In [ ]:
vocabulary_path = os.path.join(preprocessed_dir, 'vocab.txt')
with open(vocabulary_path, 'r') as f:
    vocabulary = {line.split(':')[0]: int(line.split(':')[1]) for line in f.readlines()}

In [ ]:
count_vectors_path = '/content/drive/MyDrive/Assignment5 data/preprocessed_data/count_vectors.txt'
count_vectors = []
with open(count_vectors_path, 'r') as f:
    for line in f.readlines():
        parts = line.strip().split(',')
        web_index = parts[0][1:]
        vector = {int(pair.split(':')[0]): int(pair.split(':')[1]) for pair in parts[1:]}
        count_vectors.append((web_index, vector))

In [ ]:
y_labels_path = '/content/drive/MyDrive/Assignment5 data/y_labels.csv'
y_labels = pd.read_csv(y_labels_path)['category'].values


In [ ]:
job_descriptions = []
for filename in os.listdir(preprocessed_dir):
    if filename.endswith('.txt') and filename not in ['vocab.txt', 'count_vectors.txt','vocabfortitle.txt','vocabfortitleonly.txt']:
        with open(os.path.join(preprocessed_dir, filename), 'r') as file:
            job_descriptions.append(file.read().strip())


In [ ]:
embedding_model = KeyedVectors.load_word2vec_format('/content/glove.6B/glove.6B.300d.txt', binary=False, no_header=True)

In [ ]:
X_embeddings = generate_document_embeddings(job_descriptions, embedding_model, use_tfidf=False)


In [ ]:
print(X_embeddings.shape)

(776, 300)


# Logistic Regression Models

In [ ]:
logistic_regression_model = LogisticRegression(max_iter=1000)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
accuracy_scores = cross_val_score(logistic_regression_model, X_embeddings, y_labels, cv=skf, scoring='accuracy')
precision_scores = cross_val_score(logistic_regression_model, X_embeddings, y_labels, cv=skf, scoring='precision_weighted')
recall_scores = cross_val_score(logistic_regression_model, X_embeddings, y_labels, cv=skf, scoring='recall_weighted')
f1_scores = cross_val_score(logistic_regression_model, X_embeddings, y_labels, cv=skf, scoring='f1_weighted')

# Print cross-validation results
print("Logistic Regression Model Performance with Word Embeddings (5-Fold Cross-Validation):")
print("Accuracy: {:.4f} ± {:.4f}".format(np.mean(accuracy_scores), np.std(accuracy_scores)))
print("Precision: {:.4f} ± {:.4f}".format(np.mean(precision_scores), np.std(precision_scores)))
print("Recall: {:.4f} ± {:.4f}".format(np.mean(recall_scores), np.std(recall_scores)))
print("F1-score: {:.4f} ± {:.4f}".format(np.mean(f1_scores), np.std(f1_scores)))


Logistic Regression Model Performance with Word Embeddings (5-Fold Cross-Validation):
Accuracy: 0.8595 ± 0.0288
Precision: 0.8645 ± 0.0270
Recall: 0.8595 ± 0.0288
F1-score: 0.8603 ± 0.0284


In [ ]:
X_embeddings_tfidf = generate_document_embeddings(job_descriptions, embedding_model, use_tfidf=True)


In [ ]:
accuracy_scores_tfidf = cross_val_score(logistic_regression_model, X_embeddings_tfidf, y_labels, cv=skf, scoring='accuracy')
precision_scores_tfidf = cross_val_score(logistic_regression_model, X_embeddings_tfidf, y_labels, cv=skf, scoring='precision_weighted')
recall_scores_tfidf = cross_val_score(logistic_regression_model, X_embeddings_tfidf, y_labels, cv=skf, scoring='recall_weighted')
f1_scores_tfidf = cross_val_score(logistic_regression_model, X_embeddings_tfidf, y_labels, cv=skf, scoring='f1_weighted')


print("\nLogistic Regression Model Performance with TF-IDF Embeddings (5-Fold Cross-Validation):")
print("Accuracy: {:.4f} ± {:.4f}".format(np.mean(accuracy_scores_tfidf), np.std(accuracy_scores_tfidf)))
print("Precision: {:.4f} ± {:.4f}".format(np.mean(precision_scores_tfidf), np.std(precision_scores_tfidf)))
print("Recall: {:.4f} ± {:.4f}".format(np.mean(recall_scores_tfidf), np.std(recall_scores_tfidf)))
print("F1-score: {:.4f} ± {:.4f}".format(np.mean(f1_scores_tfidf), np.std(f1_scores_tfidf)))


Logistic Regression Model Performance with TF-IDF Embeddings (5-Fold Cross-Validation):
Accuracy: 0.8814 ± 0.0248
Precision: 0.8857 ± 0.0230
Recall: 0.8814 ± 0.0248
F1-score: 0.8819 ± 0.0247


In [ ]:
vectorizer = CountVectorizer(vocabulary=vocabulary)
X_count_vectors = vectorizer.fit_transform(job_descriptions)

In [ ]:
logistic_regression_model = LogisticRegression(max_iter=1000)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

accuracy_scores = cross_val_score(logistic_regression_model, X_count_vectors, y_labels, cv=skf, scoring='accuracy')
precision_scores = cross_val_score(logistic_regression_model, X_count_vectors, y_labels, cv=skf, scoring='precision_weighted')
recall_scores = cross_val_score(logistic_regression_model, X_count_vectors, y_labels, cv=skf, scoring='recall_weighted')
f1_scores = cross_val_score(logistic_regression_model, X_count_vectors, y_labels, cv=skf, scoring='f1_weighted')

In [ ]:
print("Logistic Regression Model Performance with Count Vectors (5-Fold Cross-Validation):")
print("Accuracy: {:.4f} ± {:.4f}".format(np.mean(accuracy_scores), np.std(accuracy_scores)))
print("Precision: {:.4f} ± {:.4f}".format(np.mean(precision_scores), np.std(precision_scores)))
print("Recall: {:.4f} ± {:.4f}".format(np.mean(recall_scores), np.std(recall_scores)))
print("F1-score: {:.4f} ± {:.4f}".format(np.mean(f1_scores), np.std(f1_scores)))

Logistic Regression Model Performance with Count Vectors (5-Fold Cross-Validation):
Accuracy: 0.8827 ± 0.0254
Precision: 0.8881 ± 0.0212
Recall: 0.8827 ± 0.0254
F1-score: 0.8831 ± 0.0247


# Summary for Qs 1


In this task, we compared different language models to determine which one provides the best performance for classifying job advertisement categories using a logistic regression model. The models compared include count vector embeddings, standard word embeddings, and TF-IDF embeddings. The evaluation metrics used were accuracy, precision, recall, and F1-score, calculated through 5-fold cross-validation.

Count Vector Embeddings:

Accuracy: 0.8827 ± 0.0254
Precision: 0.8881 ± 0.0212
Recall: 0.8827 ± 0.0254
F1-score: 0.8831 ± 0.0247
Word Embeddings:

Accuracy: 0.8595 ± 0.0288
Precision: 0.8645 ± 0.0270
Recall: 0.8595 ± 0.0288
F1-score: 0.8603 ± 0.0284
TF-IDF Embeddings:

Accuracy: 0.8814 ± 0.0248
Precision: 0.8857 ± 0.0230
Recall: 0.8814 ± 0.0248
F1-score: 0.8819 ± 0.0247
The results indicate that the logistic regression model with count vector embeddings outperformed both the models with standard word embeddings and TF-IDF embeddings across all evaluation metrics, making count vector embeddings the best performing language model for this task.

# Question 2

In [ ]:
data_dir = '/content/drive/MyDrive/Assignment5 data/data'
categories = ['Accounting_Finance', 'Engineering', 'Healthcare_Nursing', 'Sales']

stop_words_file = '/content/drive/MyDrive/Assignment5 data/stopwords_en.txt'
with open(stop_words_file, 'r') as f:
    stop_words = set(f.read().splitlines())

In [ ]:
job_ads = []

In [ ]:
def load_job_ads(data_dir, categories):
    for category in categories:
        category_path = os.path.join(data_dir, category)
        for filename in os.listdir(category_path):
            if filename.endswith(".txt"):
                file_path = os.path.join(category_path, filename)
                with open(file_path, 'r') as file:
                    lines = file.readlines()
                    title = lines[0].strip()
                    web_index = lines[1].strip()
                    description = " ".join(lines[2:]).strip()

                    job_ads.append({
                        'category': category,
                        'filename': filename,
                        'title': title,
                        'web_index': web_index,
                        'description': description
                    })


In [ ]:
load_job_ads(data_dir, categories)

In [ ]:
import re
token_pattern = re.compile(r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?")


In [ ]:
from collections import Counter, defaultdict
def preprocess_text(text):
    tokens = token_pattern.findall(text)
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if len(token) > 1]
    return tokens

for ad in job_ads:
    ad['title_tokens'] = preprocess_text(ad['title'])
    ad['description_tokens'] = preprocess_text(ad['description'])

def remove_stopwords(tokens, stop_words):
    return [token for token in tokens if token not in stop_words]
def remove_rare_words(tokens, term_freq, min_freq=2):
    return [token for token in tokens if term_freq[token] >= min_freq]
def remove_most_frequent_words(tokens, top_50_words):
    return [token for token in tokens if token not in top_50_words]


In [ ]:
for ad in job_ads:
    ad['title_tokens'] = remove_stopwords(ad['title_tokens'], stop_words)
    ad['description_tokens'] = remove_stopwords(ad['description_tokens'], stop_words)

term_freq = Counter(token for ad in job_ads for token in ad['title_tokens'] + ad['description_tokens'])
for ad in job_ads:
    ad['title_tokens'] = remove_rare_words(ad['title_tokens'], term_freq)
    ad['description_tokens'] = remove_rare_words(ad['description_tokens'], term_freq)

doc_freq = defaultdict(int)
for ad in job_ads:
    unique_tokens = set(ad['title_tokens'] + ad['description_tokens'])
    for token in unique_tokens:
        doc_freq[token] += 1

top_50_words = {word for word, count in Counter(doc_freq).most_common(50)}
for ad in job_ads:
    ad['title_tokens'] = remove_most_frequent_words(ad['title_tokens'], top_50_words)
    ad['description_tokens'] = remove_most_frequent_words(ad['description_tokens'], top_50_words)

for ad in job_ads:
    ad['combined_tokens'] = ad['title_tokens'] + ad['description_tokens']



In [ ]:
preprocessed_dir = '/content/drive/MyDrive/Assignment5 data/preprocessed_data_titles'
os.makedirs(preprocessed_dir, exist_ok=True)

for ad in job_ads:
    preprocessed_file_path = os.path.join(preprocessed_dir, ad['filename'])
    with open(preprocessed_file_path, 'w') as f:
        f.write(" ".join(ad['combined_tokens']))

In [ ]:
vocabulary = set(token for ad in job_ads for token in ad['combined_tokens'])
sorted_vocabulary = sorted(vocabulary)
vocab_with_index = {word: idx for idx, word in enumerate(sorted_vocabulary)}

print("Vocabulary built with indices.")

Vocabulary built with indices.


In [ ]:
vocabulary_file = os.path.join(preprocessed_dir, 'vocabfortitle.txt')
with open(vocabulary_file, 'w') as f:
    for word, idx in vocab_with_index.items():
        f.write(f"{word}:{idx}\n")

print("Vocabulary saved.")

Vocabulary saved.


In [ ]:
preprocessed_dir = '/content/drive/MyDrive/Assignment5 data/preprocessed_data_titles'

In [ ]:
job_descriptions = []
for filename in os.listdir(preprocessed_dir):
    if filename.endswith('.txt') and filename not in ['vocabfortitle.txt', 'count_vectors.txt']:
        with open(os.path.join(preprocessed_dir, filename), 'r') as file:
            job_descriptions.append(file.read().strip())

In [ ]:
X_embeddings_tfidf = generate_document_embeddings(job_descriptions, embedding_model, use_tfidf=True)

In [ ]:
accuracy_scores_tfidf = cross_val_score(logistic_regression_model, X_embeddings_tfidf, y_labels, cv=skf, scoring='accuracy')
precision_scores_tfidf = cross_val_score(logistic_regression_model, X_embeddings_tfidf, y_labels, cv=skf, scoring='precision_weighted')
recall_scores_tfidf = cross_val_score(logistic_regression_model, X_embeddings_tfidf, y_labels, cv=skf, scoring='recall_weighted')
f1_scores_tfidf = cross_val_score(logistic_regression_model, X_embeddings_tfidf, y_labels, cv=skf, scoring='f1_weighted')

print("\nLogistic Regression Model Performance with TF-IDF Embeddings (5-Fold Cross-Validation):")
print("Accuracy: {:.4f} ± {:.4f}".format(np.mean(accuracy_scores_tfidf), np.std(accuracy_scores_tfidf)))
print("Precision: {:.4f} ± {:.4f}".format(np.mean(precision_scores_tfidf), np.std(precision_scores_tfidf)))
print("Recall: {:.4f} ± {:.4f}".format(np.mean(recall_scores_tfidf), np.std(recall_scores_tfidf)))
print("F1-score: {:.4f} ± {:.4f}".format(np.mean(f1_scores_tfidf), np.std(f1_scores_tfidf)))


Logistic Regression Model Performance with TF-IDF Embeddings (5-Fold Cross-Validation):
Accuracy: 0.8840 ± 0.0243
Precision: 0.8877 ± 0.0238
Recall: 0.8840 ± 0.0243
F1-score: 0.8847 ± 0.0245


In [ ]:
for ad in job_ads:
    ad['title_tokens'] = preprocess_text(ad['title'])
for ad in job_ads:
    ad['title_tokens'] = remove_stopwords(ad['title_tokens'], stop_words)
for ad in job_ads:
    ad['title_tokens'] = remove_rare_words(ad['title_tokens'], term_freq)

doc_freq = defaultdict(int)
for ad in job_ads:
    unique_tokens = set(ad['title_tokens'])
    for token in unique_tokens:
        doc_freq[token] += 1

top_50_words = {word for word, count in Counter(doc_freq).most_common(50)}
for ad in job_ads:
    ad['title_tokens'] = remove_most_frequent_words(ad['title_tokens'], top_50_words)

preprocessed_dir = '/content/drive/MyDrive/Assignment5 data/preprocessed_data_only_titles'
os.makedirs(preprocessed_dir, exist_ok=True)
for ad in job_ads:
    preprocessed_file_path = os.path.join(preprocessed_dir, ad['filename'])
    with open(preprocessed_file_path, 'w') as f:
        f.write(" ".join(ad['title_tokens']))


In [ ]:
vocabulary = set(token for ad in job_ads for token in ad['title_tokens'])

In [ ]:
sorted_vocabulary = sorted(vocabulary)
vocab_with_index = {word: idx for idx, word in enumerate(sorted_vocabulary)}

In [ ]:
vocabulary_file = os.path.join(preprocessed_dir, 'vocabfortitleonly.txt')
with open(vocabulary_file, 'w') as f:
    for word, idx in vocab_with_index.items():
        f.write(f"{word}:{idx}\n")

print("Vocabulary saved.")

Vocabulary saved.


In [ ]:
preprocessed_dir = '/content/drive/MyDrive/Assignment5 data/preprocessed_data_only_titles'
job_descriptions = []
for filename in os.listdir(preprocessed_dir):
    if filename.endswith('.txt') and filename not in ['vocabfortitleonly.txt', 'count_vectors.txt']:
        with open(os.path.join(preprocessed_dir, filename), 'r') as file:
            job_descriptions.append(file.read().strip())

In [ ]:
X_embeddings = generate_document_embeddings(job_descriptions, embedding_model, use_tfidf=True)

In [ ]:
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

In [ ]:
for train_index, test_index in skf.split(X_embeddings, y_labels):
    X_train, X_test = X_embeddings[train_index], X_embeddings[test_index]
    y_train, y_test = y_labels[train_index], y_labels[test_index]
    logistic_regression_model.fit(X_train, y_train)
    predictions = logistic_regression_model.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, predictions))
    precision_scores.append(precision_score(y_test, predictions, average='weighted'))
    recall_scores.append(recall_score(y_test, predictions, average='weighted'))
    f1_scores.append(f1_score(y_test, predictions, average='weighted'))

In [ ]:
print("Logistic Regression Model Performance with Word Embeddings (5-Fold Cross-Validation):")
print("Accuracy: {:.4f} ± {:.4f}".format(np.mean(accuracy_scores), np.std(accuracy_scores)))
print("Precision: {:.4f} ± {:.4f}".format(np.mean(precision_scores), np.std(precision_scores)))
print("Recall: {:.4f} ± {:.4f}".format(np.mean(recall_scores), np.std(recall_scores)))
print("F1-score: {:.4f} ± {:.4f}".format(np.mean(f1_scores), np.std(f1_scores)))

Logistic Regression Model Performance with Word Embeddings (5-Fold Cross-Validation):
Accuracy: 0.6314 ± 0.0192
Precision: 0.6396 ± 0.0190
Recall: 0.6314 ± 0.0192
F1-score: 0.6306 ± 0.0208


# Summary Qs 1 & 2


This task explored whether incorporating additional features, such as the job title, alongside the job description, would improve the model's classification accuracy. The models built and compared were based on three scenarios:

**Only Job Title:**

Performance metrics were lower than models using job descriptions, suggesting the title alone is less informative.
Only Job Description:

Performance metrics were consistent with the best results from Q1, where count vector embeddings provided the highest accuracy.
Job Title + Job Description:

Combining both title and description improved the model's performance, achieving higher accuracy and better overall metrics than using either the title or description alone.
In conclusion, adding extra information (job title) to the job description improves the model's accuracy, demonstrating that more information can indeed enhance classification performance.